In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax
import os
#os.environ["XLA_FLAGS"] = '--xla_force_host_platform_device_count=16'
jax.devices()

[cuda(id=0), cuda(id=1), cuda(id=2), cuda(id=3)]

In [3]:
import pandas as pd
from src import ComputationalHypergraphDiscovery as CHD
#from ComputationalHypergraphDiscovery.Modes import LinearMode, QuadraticMode, GaussianMode
import numpy as onp
import matplotlib.pyplot as plt
import networkx as nx
import pickle

# Linear FPUT model

In [4]:
df=pd.read_csv('./examples/FPUT_linear.csv')
df

,$X_{0}$,$X_{1}$,$X_{2}$,$X_{3}$,$X_{4}$,$X_{5}$,$X_{6}$,$X_{7}$,$X_{8}$,$X_{9}$,...,$\ddot{X}_{0}$,$\ddot{X}_{1}$,$\ddot{X}_{2}$,$\ddot{X}_{3}$,$\ddot{X}_{4}$,$\ddot{X}_{5}$,$\ddot{X}_{6}$,$\ddot{X}_{7}$,$\ddot{X}_{8}$,$\ddot{X}_{9}$
0,0.200543,-0.268028,0.483134,-0.105517,-0.074423,0.334300,-0.187482,0.029494,-0.138106,-0.001547,...,-54.198255,98.798327,-108.524794,50.199359,30.587862,-75.370803,59.839341,-31.150640,24.636881,-10.935974
1,-0.017160,0.152926,0.149011,-0.115220,0.002534,-0.062572,-0.163796,-0.093034,-0.052793,0.110310,...,15.166878,-14.094042,-21.085597,30.940767,-14.811615,-2.925602,13.930864,-2.472201,9.951854,-22.146463
2,0.106949,-0.202711,0.473624,-0.130464,-0.052804,0.383457,-0.222962,-0.014684,-0.105294,-0.017773,...,-33.745320,79.865593,-103.714248,55.221631,29.046586,-84.456997,65.990359,-24.209833,14.428580,-5.649607
3,-0.004821,0.306659,-0.309887,0.047212,-0.022266,-0.016344,-0.012491,-0.053466,0.451172,-0.311484,...,25.620459,-75.170161,78.865289,-34.552810,6.107416,-0.167508,-3.631148,44.194637,-102.650720,87.005270
4,-0.124585,0.056731,-0.379151,0.143918,0.150544,-0.173367,0.176687,0.145475,-0.041774,-0.064768,...,24.778037,-49.993023,77.674934,-41.831810,-26.773515,54.591217,-30.882615,-12.639003,13.304734,7.108632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.146389,0.006213,0.403906,-0.256289,0.257063,-0.204374,0.420246,-0.265031,0.169803,0.006959,...,-23.211749,43.567359,-85.688913,95.057294,-78.957921,87.970667,-106.101685,90.728989,-48.411955,12.626741
996,0.198881,0.128679,-0.316517,0.254229,-0.527290,0.141267,-0.251007,-0.111275,-0.334562,0.488099,...,-21.795657,-30.374574,82.291305,-109.533434,117.456120,-85.927293,43.092495,-29.404581,84.721841,-106.171611
997,-0.192739,-0.083689,0.311481,-0.068822,-0.134921,-0.426854,0.014337,-0.195667,0.027542,-0.031051,...,24.444986,23.175640,-62.813320,25.450580,-18.292596,59.383076,-52.746789,35.090210,-22.825899,7.261119
998,-0.154976,0.074445,-0.066226,-0.182642,0.284772,0.080398,-0.274946,0.027231,0.325166,-0.052136,...,31.136173,-29.977377,1.964557,47.290251,-54.414828,-12.228503,53.259083,-0.343514,-54.694186,34.784423


In [17]:

graph_discovery=CHD.GraphDiscovery.from_dataframe(df,normalize=True,kernels=[CHD.Modes.LinearMode(),CHD.Modes.QuadraticMode()])

In [ ]:
import pynvml

pynvml.nvmlInit()

device_count = pynvml.nvmlDeviceGetCount()
for i in range(device_count):
    handle = pynvml.nvmlDeviceGetHandleByIndex(i)
    info = pynvml.nvmlDeviceGetMemoryInfo(handle)
    print(f"Device {i}: {pynvml.nvmlDeviceGetName(handle)}")
    print(f"Total memory: {info.total / 1024 ** 2:.2f} MiB")
    print(f"Used memory: {info.used / 1024 ** 2:.2f} MiB")
    print(f"Free memory: {info.free / 1024 ** 2:.2f} MiB")

pynvml.nvmlShutdown()


Device 0: Tesla V100-DGXS-32GB
Total memory: 32768.00 MiB
Used memory: 25080.94 MiB
Free memory: 7687.06 MiB
Device 1: Tesla V100-DGXS-32GB
Total memory: 32768.00 MiB
Used memory: 593.62 MiB
Free memory: 32174.38 MiB
Device 2: Tesla V100-DGXS-32GB
Total memory: 32768.00 MiB
Used memory: 593.62 MiB
Free memory: 32174.38 MiB
Device 3: Tesla V100-DGXS-32GB
Total memory: 32768.00 MiB
Used memory: 593.62 MiB
Free memory: 32174.38 MiB


In [18]:
graph_discovery.fit()

start
linear
quadratic


Finding ancestors with kernel [linear]: 100%|██████████| 28/28 [00:03<00:00,  7.32it/s]


> /home/theo/CHD/ComputationalHypergraphDiscovery/src/ComputationalHypergraphDiscovery/_GraphDiscoveryMain.py(510)process_results()
    508             mask,
    509             *kernel_performance,
--> 510         ) in zip(
    511             targets,
    512             mask_kernel,

[]
Kernel [linear] has n/(n+s)=9.868449759245921e-13, Z=(1.00, 1.00), gamma=1.00e-09
Kernel [quadratic] has n/(n+s)=1.356019706091731e-12, Z=(1.00, 1.00), gamma=1.00e-09
$X_{0}$ has ancestors with the kernel [linear] | (n/(s+n)=0.00)


IndexError: list index out of range

In [ ]:
graph_discovery.plot_graph(type_label=False)

In [ ]:
def plot_a_network(G,type_label=True):
    pos=nx.kamada_kawai_layout(G, G.nodes())
    nx.draw(G, with_labels=True, pos=pos, node_size=600, font_size=8, alpha=0.6)
    if type_label:
        nx.draw_networkx_edge_labels(G,pos,edge_labels=nx.get_edge_attributes(G,'type'))
    x_values, y_values = zip(*pos.values())
    x_max = max(x_values)
    x_min = min(x_values)
    x_margin = (x_max - x_min) * 0.25
    plt.xlim(x_min - x_margin, x_max + x_margin)
G2=nx.subgraph(graph_discovery.G,[x for x in graph_discovery.G.nodes() if '\dot' not in x])
plot_a_network(G2,type_label=False)

# Non-Linear FPUT

In [ ]:
df_non_linear=pd.read_csv('./examples/FPUT_non_linear.csv')
df_non_linear

In [ ]:

graph_discovery_non_linear=CHD.GraphDiscovery.from_dataframe(df_non_linear,normalize=True)

In [ ]:
graph_discovery_non_linear.fit()

In [ ]:
graph_discovery_non_linear.plot_graph(type_label=False)

In [ ]:
def plot_a_network(G,type_label=True):
    pos=nx.kamada_kawai_layout(G, G.nodes())
    nx.draw(G, with_labels=True, pos=pos, node_size=600, font_size=8, alpha=0.6)
    if type_label:
        nx.draw_networkx_edge_labels(G,pos,edge_labels=nx.get_edge_attributes(G,'type'))
    x_values, y_values = zip(*pos.values())
    x_max = max(x_values)
    x_min = min(x_values)
    x_margin = (x_max - x_min) * 0.25
    plt.xlim(x_min - x_margin, x_max + x_margin)
G2=nx.subgraph(graph_discovery_non_linear.G,[x for x in graph_discovery.G.nodes() if '\dot' not in x])
plot_a_network(G2,type_label=False)

In [ ]:
plot_a_network(nx.ego_graph(G2.reverse(),'$\ddot{X}_{1}$',undirected=False,radius=1).reverse(),type_label=True)

In [ ]:
plot_a_network(nx.ego_graph(graph_discovery.G.reverse(),'$\dot{X}_{5}$',undirected=False,radius=1).reverse(),type_label=False)